## Import der benötigten Bibliotheken

In [ ]:
import pprint
import nltk
import csv
import operator
import pandas    as pd
from pymongo     import MongoClient
from nltk        import word_tokenize
from nltk.corpus import stopwords

## Laden der Stopwords
Die 1. Zelle muss nur beim ersten Mal ausgeführt werden

In [ ]:
nltk.download('stopwords')

In [ ]:
stop = set(stopwords.words('german'))
stop.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'])

## Konvertierung der .txt-Datenquellen in .csv-Dateien
Muss nur beim ersten Mal ausgeführt werden

In [ ]:
sentences_csv = pd.read_csv ('deu_news_2015_3M-sentences.txt', header = None, delimiter='\t')
sentences_csv.to_csv ('sentences_de.csv', index=None)

## Entfernen der Stopwords

In [ ]:
def remove_stopwords(sentences):
    queue = []
    for sentence in sentences:
        result = ' '.join([i for i in sentence.lower().split() if i not in stop]) #Habe hier eine Liste statt einer Menge genommen, da sonst die Reihenfolge geändert wird
        queue.append(result)
    return queue

In [ ]:
nostop = remove_stopwords(sentences_csv[1]) #(sentences_csv[1][100:200]) Für Test nur 100 Datensätze verwenden
df = pd.DataFrame(nostop)
df.to_csv('sentences_preprocessed.csv', index=False)

## Verbindung mit mongoDB

In [ ]:
connection_string      = "mongodb://localhost:27017"
db                     = MongoClient(connection_string)
sentences              = db.dw.sentences
sentences_preprocessed = db.dw.sentences_preprocessed

## Sentences und Stopwords der Database in mongoDB hinzufügen

`read_csv()` liest eine .csv-Datei ein und gibt ein Dictionary zurück, dass dann mit `save_to_mongo()` in einer Collection gespeichert wird. Dafür kann zunächst die Collection gelöscht werden (`drop_collection = True`). Sonst werden die Daten zusätzlich in die Collection eingefügt.

In [ ]:
def read_csv(path, columns):
    reader = open(path, 'r', encoding='utf-8')
    return csv.DictReader(reader, columns)   

In [ ]:
def save_to_mongo(collection, path, columns, drop_collection):
    if drop_collection:
        collection.drop()
    data = read_csv(path, columns)
    result = collection.insert_many(data)
    print('%d rows are saved to "%s" collection successfully!' % (len(result.inserted_ids), collection.name))

In [ ]:
save_to_mongo(sentences, 'sentences_de.csv', ('id','sentence'), True)

In [ ]:
save_to_mongo(sentences_preprocessed, 'sentences_preprocessed.csv', ('s'), True)

Wenn ich die als Column-Name gleich `sentence` genommen habe, dann hat er daraus eine Spalte `s`, eine `e`, eine `n` und so weiter gemacht...Deswegen hier mit der späteren Umbenennung.

In [ ]:
sentences_preprocessed.update_many( {}, { '$rename': { "s": "sentence" } } )

In [ ]:
test_data = nostop[:2000000]

In [ ]:
def count_bigrams(data):
    bigram_dict = dict()
    for text in data:
        bigrams = list(nltk.bigrams(text.split()))
        for bigram in bigrams:
            if bigram in bigram_dict:
                bigram_dict[bigram] += 1
            else:
                bigram_dict[bigram] = 1
    return bigram_dict

In [ ]:
def get_k_most_frequent(dictionary, k):
    sorted_dictionary =  dict(sorted(dictionary.items(), key=operator.itemgetter(1),reverse=True))
    result = sorted_dictionary.items()
    return list(result)[:k]

In [ ]:
def pretty_print_most_frequent(data):
    for i in range(len(data)):
        words, frequency = data[i]
        print('Platz ', i+1, ': Die Wortkombination "', ' '.join(words), \
              '" kommt in den Daten ', frequency, ' mal vor.', sep='')

In [ ]:
def get_k_most_frequent_bigrams(data, k):
    bigram_dict   = count_bigrams(data)
    most_frequent = get_k_most_frequent(bigram_dict, k)
    pretty_print_most_frequent(most_frequent)

In [ ]:
get_k_most_frequent_bigrams(test_data, 10)

## ToDos: 
- Einfügen von sentences_preprocessed in DB ohne Umweg über DataFrame und .csv
- Spaltenname ohne Umbenennung
- Daten für Analyse aus DB ziehen
- Das Ergebnis (komplettes Bigram-Dict?) in der DB speichern
- Weitere Analysen?